***Parte B***: Redes Neuronales Convolucionales

In [ ]:
pip install torch torchvision matplotlib

***Preguntas***
1. ¿Que significa que el batch_size sea 4? Explica cómo afectaría al cálculo de los pesos en cada epoch.
La variable batch_size define cuantos datos se procesan en cada lote, al tener un valor pequeño el calculo de los pesos se realiza más veces en cada epoch.

2. ¿Por qué al dividir el conjunto de entrenamiento el parámetro shuffle está a True, mientras que el conjunto de test está a False?
El parametro shuffle reorganiza de manera aleatoria los datos que se cargan en cada epoch, esto puede ser beneficioso para evitar patrones que puedan afectar al desempeño del modelo, en el test no es necesario ya que son los datos de validación y por lo tanto no es necesario hacer el shuffle.

3. Ver en la celda de código adyacente.

4. Indica la línea de código donde se realiza el cálculo de los gradientes para determinar cómo
modificar los pesos (los filtros kernel), e indica la línea de código donde se recalculan los pesos.
¿Qué se ha tenido que realizar antes para poder hacer el cálculo de gradientes y la
actualización de pesos?

5. 


In [ ]:
'''
Pregunta 3
'''
class Net(nn.Module):  # Define una clase para la red neuronal que hereda de nn.Module
    def __init__(self):  # Constructor de la clase
        super().__init__()  # Llama al constructor de la clase base nn.Module
        # Primera capa convolucional: toma imágenes con 3 canales (RGB), aplica 6 filtros de tamaño 5x5
        self.conv1 = nn.Conv2d(3, 6, 5)  
        # Capa de pooling: aplica max pooling con un tamaño de ventana 2x2 y un paso (stride) de 2
        self.pool = nn.MaxPool2d(2, 2)  
        # Segunda capa convolucional: toma 6 canales de entrada y aplica 16 filtros de tamaño 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)  
        # Primera capa completamente conectada: conecta 16 * 5 * 5 entradas (salida de la convolución) a 120 neuronas
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  
        # Segunda capa completamente conectada: conecta 120 neuronas a 84 neuronas
        self.fc2 = nn.Linear(120, 84)  
        # Tercera capa completamente conectada: conecta 84 neuronas a 10 neuronas (una por cada clase)
        self.fc3 = nn.Linear(84, 10)  

    def forward(self, x):  # Define el flujo de datos a través de la red (forward propagation)
        # Aplica la primera capa convolucional, seguida de ReLU y max pooling
        x = self.pool(F.relu(self.conv1(x)))  
        # Aplica la segunda capa convolucional, seguida de ReLU y max pooling
        x = self.pool(F.relu(self.conv2(x)))  
        # Aplana el tensor (convierte las dimensiones de características en una sola dimensión, excepto el batch)
        x = torch.flatten(x, 1)  # Mantiene la dimensión del batch (dimensión 0)
        # Aplica la primera capa completamente conectada con activación ReLU
        x = F.relu(self.fc1(x))  
        # Aplica la segunda capa completamente conectada con activación ReLU
        x = F.relu(self.fc2(x))  
        # Aplica la última capa completamente conectada (sin activación, ya que es la salida final)
        x = self.fc3(x)  
        return x  # Devuelve la salida final (logits para las 10 clases)

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

#Declaramos las dos redes neuronales que vamos a utilizar
class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        # Primera capa convolucional: 3 canales de entrada (RGB), 8 filtros, tamaño de kernel 3x3
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)  # Capa de pooling: tamaño 2x2, stride 2
        # Segunda capa convolucional: 8 canales de entrada, 20 filtros, tamaño de kernel 3x3
        self.conv2 = nn.Conv2d(20, 20, kernel_size=5)
        # Ajuste de la capa completamente conectada para las nuevas dimensiones
        self.fc1 = nn.Linear(20 * 5 * 5, 120)  # 20 filtros, tamaño 5x5 después de convoluciones y pooling
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Primera capa convolucional + ReLU + pooling
        x = self.pool(F.relu(self.conv2(x)))  # Segunda capa convolucional + ReLU + pooling
        x = torch.flatten(x, 1)  # Aplanar todas las dimensiones excepto el batch
        x = F.relu(self.fc1(x))  # Primera capa completamente conectada + ReLU
        x = F.relu(self.fc2(x))  # Segunda capa completamente conectada + ReLU
        x = self.fc3(x)  # Capa de salida
        return x
    
  
net1 = Net1()
net2 = Net2()

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer1 = optim.SGD(net1.parameters(), lr=0.001, momentum=0.9)
optimizer2 = optim.SGD(net2.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer1.zero_grad()

        # forward + backward + optimize
        outputs = net1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer1.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer2.zero_grad()

        # forward + backward + optimize
        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer2.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')
